In [1]:
import Pkg; Pkg.add("JuMP");  Pkg.add("GLPK");  Pkg.add("DelimitedFiles"); 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
using JuMP          # Modeling language for optimization
using GLPK          # Open-source solver for linear programming

In [3]:
# Dados do problema
n = 42  # número de operações
m = 10  # número de trabalhadores

10

In [4]:
production_times = [
    0.100000 0.157916 1.262799 1.290917 0.109419 0.956469 1.048012 0.267745 0.616112 0.381447 0.142017 0.100000 0.100000 0.275619 1.067472 0.116311 0.043384 0.100000 0.753886 0.477781 0.100000 1.404547 0.127177 1.373362 0.714355 0.217598 0.309765 1.207631 1.205433 0.130243 1.219805 0.335577 0.589780 1.709745 0.139897 0.626678 1.334373 0.803798 0.700504 0.466742 0.722569 0.557902 
    0.988762 0.026033 0.344655 0.603306 0.587490 0.652222 0.421817 0.088780 0.852775 0.218461 0.801375 0.222353 0.549195 0.600392 0.637823 0.544367 0.610101 0.234584 0.665633 0.691910 0.571449 0.589709 0.100000 0.286885 0.100000 0.436417 0.730759 1.207174 0.985311 0.151022 0.792664 0.817342 0.778123 0.994229 0.494307 0.405089 0.169122 0.752759 0.663545 0.496304 0.122936 0.876286 
    0.572378 0.100000 0.011968 0.950888 0.072999 0.812977 0.084497 0.427929 0.772320 0.616919 0.185853 0.573961 0.339514 0.353842 0.722161 0.100000 0.100000 0.069952 0.628917 0.005640 0.100000 0.367631 0.100000 1.223211 0.645359 0.100000 0.281969 0.281988 0.699690 0.100000 0.850426 0.498823 1.022732 0.998399 0.284923 1.037996 0.756077 0.412065 0.914034 0.520254 0.387817 0.610792 
    0.068620 0.014490 0.055620 0.100000 0.583422 0.199094 0.657882 0.348501 0.614044 0.754357 0.147757 0.100000 0.015072 0.435123 1.035354 0.100000 0.194400 0.172988 0.100000 0.343841 0.100000 0.022228 0.100000 1.509331 0.815410 0.322853 0.571434 0.676112 0.945204 0.386271 0.394986 0.983189 0.283517 0.628565 0.145346 0.191645 0.412015 0.882287 0.877377 1.424677 0.385175 0.505852 
    0.797531 0.399259 0.203260 0.814548 0.465033 0.966060 0.072941 0.749886 0.531740 1.046584 0.100000 0.100000 0.465523 0.909956 0.815424 0.431637 0.387499 0.037826 0.891858 0.025489 0.100000 0.996827 0.177246 0.673288 0.100000 0.352558 0.450197 1.442929 0.001378 0.150140 0.674537 0.100000 0.124727 1.084218 0.100000 0.757676 0.752393 0.100000 1.530032 0.682603 1.550781 0.572838 
    0.844141 0.100000 0.261215 0.306577 0.983179 0.275689 0.715033 0.299600 0.063611 0.678545 0.057644 0.100000 0.399239 1.384731 0.445457 0.314536 0.100000 0.428565 0.175239 0.591247 0.152156 0.653230 0.202500 0.674880 0.652734 0.625663 0.986547 0.543733 0.474079 0.412397 1.076239 0.669097 0.650080 0.927165 0.333914 1.090906 0.202114 0.303397 1.474339 0.683823 0.221622 0.575815 
    0.535806 0.221485 0.100000 0.402019 0.235406 0.618712 0.345675 0.010342 1.003911 0.374969 0.420879 0.100000 0.267348 1.104489 0.100000 0.143550 0.100000 0.100000 0.106673 0.243619 0.100000 0.511684 0.269377 0.495636 0.710855 0.142100 0.597025 0.304712 0.280579 0.502231 0.774837 0.945471 0.985205 0.226248 0.593024 0.404204 0.100000 0.877897 0.315674 0.570285 0.964077 0.897042 
    0.183328 0.736867 0.100000 0.100000 1.325092 0.100000 0.105104 1.338662 0.669275 1.209392 0.527939 0.018967 0.108079 0.544422 1.794255 0.697402 0.472250 0.228696 1.347705 0.306829 0.100000 0.885774 0.774910 0.430882 0.100000 0.100000 0.100000 0.100000 0.438878 0.100000 0.781502 1.475757 0.100000 0.484092 0.297532 0.374247 0.100000 1.056941 0.972823 0.352911 0.100000 0.575560 
    0.466190 0.509117 0.325959 0.372250 0.100000 0.249220 0.100000 0.382986 0.505143 0.408174 0.205062 0.162168 0.257930 0.743567 0.672855 0.346580 0.820722 0.164462 0.454791 0.552017 0.252859 1.004156 0.280069 1.028491 0.646793 0.353609 0.290439 0.263874 0.656616 0.100000 0.885402 0.725558 0.824257 0.522019 0.675642 0.684066 0.865765 0.408710 1.021669 0.632090 0.575717 0.691220 
    0.873186 0.882416 0.188929 0.295729 0.359003 0.597654 1.232744 0.001473 0.694448 1.028069 0.464547 0.577962 0.100000 0.770422 0.311185 0.638233 0.100000 0.678897 0.550846 0.358884 0.513845 0.400113 0.278348 0.528436 1.246539 0.100000 0.855153 0.377221 0.178270 0.138585 1.657865 0.966718 0.219303 0.100000 0.100000 0.961428 0.895742 0.453777 1.075138 0.175247 0.387570 0.191486 
]

10×42 Matrix{Float64}:
 0.1       0.157916  1.2628    1.29092   …  0.466742  0.722569  0.557902
 0.988762  0.026033  0.344655  0.603306     0.496304  0.122936  0.876286
 0.572378  0.1       0.011968  0.950888     0.520254  0.387817  0.610792
 0.06862   0.01449   0.05562   0.1          1.42468   0.385175  0.505852
 0.797531  0.399259  0.20326   0.814548     0.682603  1.55078   0.572838
 0.844141  0.1       0.261215  0.306577  …  0.683823  0.221622  0.575815
 0.535806  0.221485  0.1       0.402019     0.570285  0.964077  0.897042
 0.183328  0.736867  0.1       0.1          0.352911  0.1       0.57556
 0.46619   0.509117  0.325959  0.37225      0.63209   0.575717  0.69122
 0.873186  0.882416  0.188929  0.295729     0.175247  0.38757   0.191486

In [5]:
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [6]:
# Variables
@variable(model, x[1:n, 1:m], Bin)  # Assignment of tasks to workers
@variable(model, b[1:n, 1:m], Bin)  # Start of a block for a worker
@variable(model, e[1:n, 1:m], Bin)  # End of a block for a worker
@variable(model, T >= 0)  # Maximum workload

T

In [7]:
@objective(model, Min, T)

T

In [8]:
for i in 1:n
    @constraint(model, sum(x[i, j] for j in 1:m) == 1)
end

In [9]:
# Contiguity constraints: Each worker has exactly one contiguous block
for j in 1:m
    # Exactly one start and one end per worker
    @constraint(model, sum(b[i, j] for i in 1:n) == 1)
    @constraint(model, sum(e[i, j] for i in 1:n) == 1)
    
    # Start position definition
    for i in 1:n
        # Handle boundary cases (i=1 and i=num_operations)
        prev_x = (i == 1) ? 0 : x[i-1, j]
        next_x = (i == n) ? 0 : x[i+1, j]
        
        # d[i,j] = 1 if x[i,j] = 1 and x[i-1,j] = 0
        @constraint(model, b[i, j] >= x[i, j] - prev_x)
        @constraint(model, b[i, j] <= x[i, j])
        @constraint(model, b[i, j] <= 1 - prev_x)
        
        # e[i,j] = 1 if x[i,j] = 1 and x[i+1,j] = 0
        @constraint(model, e[i, j] >= x[i, j] - next_x)
        @constraint(model, e[i, j] <= x[i, j])
        @constraint(model, e[i, j] <= 1 - next_x)
    end
end

In [10]:
# Workload constraints: T >= sum of production times for each worker
for j in 1:m
    @constraint(model, T >= sum(production_times[j, i] * x[i, j] for i in 1:n))
end

In [11]:
optimize!(model)

In [12]:
println("Maximum workload (T): ", objective_value(model))

# Print assignments
for j in 1:m
    assigned_tasks = [i for i in 1:n if value(x[i, j]) > 0.5]
    if !isempty(assigned_tasks)
        println("Worker $j: Tasks ", assigned_tasks, " | Workload: ", sum(production_times[j, i] for i in assigned_tasks))
    end
end

Maximum workload (T): 1.3260990000000001
Worker 1: Tasks [10, 11, 12, 13, 14] | Workload: 0.9990829999999999
Worker 2: Tasks [22, 23, 24, 25] | Workload: 1.076594
Worker 3: Tasks [38, 39] | Workload: 1.3260990000000001
Worker 4: Tasks [1, 2, 3, 4, 5, 6] | Workload: 1.021246
Worker 5: Tasks [29, 30, 31, 32, 33] | Workload: 1.0507819999999999
Worker 6: Tasks [7, 8, 9] | Workload: 1.078244
Worker 7: Tasks [15, 16, 17, 18, 19, 20, 21] | Workload: 0.893842
Worker 8: Tasks [34, 35, 36, 37] | Workload: 1.2558710000000002
Worker 9: Tasks [26, 27, 28] | Workload: 0.9079219999999999
Worker 10: Tasks [40, 41, 42] | Workload: 0.754303
